In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, utils, backend as K
from tensorflow.keras.models import load_model

import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler


In [2]:
def tanh(x):
    return np.tanh(x)

# Function to perform predictions using the extracted parameters
def custom_predict(input_data, model):
    # Extract model parameters (weights and biases) for each layer
    model_parameters = []
    for layer in model.layers:
        layer_weights = layer.get_weights()
        model_parameters.append(layer_weights)
        
    for i, layer_params in enumerate(model_parameters):
        weights, biases = layer_params
        # Compute the linear transformation (z) for this layer
        z = np.dot(input_data, weights) + biases
        # Apply the 'tanh' activation function for all layers except the last one
        if i < len(model_parameters) - 1:
            output_data = tanh(z)
        else:
            # For the last layer, do not apply the 'tanh' activation function
            output_data = z
        # Set the output of this layer as the input for the next layer
        input_data = output_data

    # The final output is the prediction
    return output_data

In [3]:
# y = y_true y_hat = y_pred
def R2(y, y_hat):
    y = tf.convert_to_tensor(value=y, dtype='float32')
    y_hat = tf.convert_to_tensor(value=y_hat, dtype='float32')
    
    ss_res = K.sum(K.square(y - y_hat)) 
    ss_tot = K.sum(K.square(y - K.mean(y))) 
    r_squared = 1 - ss_res / (ss_tot + K.epsilon())
    
    return r_squared.numpy().astype(np.float32)  # Convert tensor to a float32 value

def RMSE(y, y_hat):
    return np.sqrt(np.mean((y_hat - y)**2))

def MAE(y, y_hat):
    return np.mean(np.abs(y - y_hat))

In [4]:
x1 = np.loadtxt('updated_new_input1.txt')
x2 = np.loadtxt('updated_new_input2.txt')
y1 = np.loadtxt('updated_new_output1.txt')
y2 = np.loadtxt('updated_new_output2.txt')

In [5]:
print(x1.shape, y1.shape, x2.shape, y2.shape)

(714230, 8) (714230, 4) (12745983, 8) (12745983, 4)


In [6]:
def test_model(model,x,y):
    evaluation = model.evaluate(x=x, y=y, verbose=0)
    print("Evaluation Result:")
    for metric in model.metrics_names:
        print(f"{metric}: {evaluation[model.metrics_names.index(metric)]:.4f}")

In [7]:
def test_model_kn_scaled(model,x,y):
    kn = 2.0e8 
    kt = 0.5 * kn
    factor = np.array([kn,kn,kt,1])
    
#     y_pred = model.predict(x, verbose=0) 
    y_pred = custom_predict(x, model)
    y_pred = y_pred * factor
    y_true = y * factor
    r2_f = R2(y_true[:,:3],y_pred[:,:3])
    r2_a = R2(y_true[:,-1],y_pred[:,-1])    
    print('force max:',      np.max(y_pred[:,:3], axis = 0),      'angle max: ', np.max(y_pred[:,-1]))
    print('true force max:', np.max(y_true[:,:3], axis = 0), 'true angle max: ', np.max(y_true[:,-1]))
    
    print('R2 force: ', r2_f, 'R2 angle: ', r2_a)
    print('RMSE force: ', RMSE(y_true[:,:3],y_pred[:,:3]), 'RMSE angle: ', RMSE(y_true[:,-1],y_pred[:,-1]))    
    print('MAE force: ', MAE(y_true[:,:3],y_pred[:,:3]), 'MAE angle: ', MAE(y_true[:,-1],y_pred[:,-1]))    

In [9]:
def test_model_1e4_scaled(model,x,y,scaler):
    
    pred_factor = np.array([1e4,1e4,1e4,1e-4])
    x = scaler.transform(x)
    
#     y_pred = model.predict(x, verbose=0) 
    y_pred = custom_predict(x, model)
    y_pred = y_pred * pred_factor
    
    kn = 2.0e8 
    kt = 0.5 * kn
    true_factor = np.array([kn,kn,kt,1])
    
    y_true = y * true_factor
    r2_f = R2(y_true[:,:3],y_pred[:,:3])
    r2_a = R2(y_true[:,-1],y_pred[:,-1])    
    
    print('force max:', np.max(y_pred[:,:3], axis = 0), 'angle max: ', np.max(y_pred[:,-1]))
    print('true force max:', np.max(y_true[:,:3], axis = 0), 'true angle max: ', np.max(y_true[:,-1]))
    
    print('R2 force: ', r2_f, 'R2 angle: ', r2_a)
    print('RMSE force: ', RMSE(y_true[:,:3],y_pred[:,:3]), 'RMSE angle: ', RMSE(y_true[:,-1],y_pred[:,-1]))    
    print('MAE force: ', MAE(y_true[:,:3],y_pred[:,:3]), 'MAE angle: ', MAE(y_true[:,-1],y_pred[:,-1]))  

In [9]:
array_10x3 = np.arange(1, 31).reshape(10, 3)
array_10x3_floats = np.arange(1.5, 31.5, 1).reshape(10, 3)

y_true = array_10x3
y_pred = array_10x3_floats

r2_f = R2(y_true[:,:3],y_pred[:,:3])
r2_a = R2(y_true[:,:-1],y_pred[:,:-1])    
print('R2 force: ', r2_f, 'R2 angle: ', r2_a)
print('RMSE force: ', RMSE(y_true[:,:3],y_pred[:,:3]), 'RMSE angle: ', RMSE(y_true[:,:-1],y_pred[:,:-1]))    
print('MAE force: ', MAE(y_true[:,:3],y_pred[:,:3]), 'MAE angle: ', MAE(y_true[:,:-1],y_pred[:,:-1]))

R2 force:  0.996663 R2 angle:  0.9966443
RMSE force:  0.5 RMSE angle:  0.5
MAE force:  0.5 MAE angle:  0.5


In [10]:
for i in range(1,9):
    
    print('test on model ', i ,'\n')
    
    model_name = 'Model_' + str(i) + '_mae_6k_new.h5'
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1')
    test_model_kn_scaled(model,x1,y1)
    print('test on dataset2')
    test_model_kn_scaled(model,x2,y2)
    print('\n')
    
    model_name = 'Model_' + str(i) + '_mse_6k_new.h5'
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1')
    test_model_kn_scaled(model,x1,y1)
    print('test on dataset2')
    test_model_kn_scaled(model,x2,y2)
    print('\n')
    
    model_name = 'Model_' + str(i) + '_mae_600k_new.h5'
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1')
    test_model_kn_scaled(model,x1,y1)
    print('test on dataset2')
    test_model_kn_scaled(model,x2,y2)
    print('\n')
    
    model_name = 'Model_' + str(i) + '_mse_600k_new.h5'
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1')
    test_model_kn_scaled(model,x1,y1)
    print('test on dataset2')
    test_model_kn_scaled(model,x2,y2)
    
    print('\n')

test on model  1 

test on dataset1
force max: [1.76252620e+08 1.18703511e+08 3.46426572e+07] angle max:  1.5815281059447595
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.5533418615598973
R2 force:  0.9936103 R2 angle:  0.9978329
RMSE force:  2053983.4027934917 RMSE angle:  0.012878974260744575
MAE force:  1027287.5057290787 MAE angle:  0.008618506962517676
test on dataset2
force max: [2.36022614e+08 1.18439468e+08 3.64819889e+07] angle max:  1.5631415415492529
true force max: [6.20017763e+08 1.17792602e+08 3.72855729e+07] true angle max:  1.5707600376421085
R2 force:  0.9982584 R2 angle:  0.99747574
RMSE force:  1099740.6071886998 RMSE angle:  0.013810898698387737
MAE force:  269311.5010334855 MAE angle:  0.0025588723417527006


test on dataset1
force max: [2.54787792e+08 1.12053471e+08 3.44567386e+07] angle max:  1.5899907085333709
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.5533418615598973
R2 force:  0.999064

RMSE force:  1027395.1434590032 RMSE angle:  0.008270534674909506
MAE force:  432711.8809834044 MAE angle:  0.0027420704173897158


test on dataset1
force max: [2.53699690e+08 1.08906486e+08 3.44309638e+07] angle max:  1.5704016242792136
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.5533418615598973
R2 force:  0.99775034 R2 angle:  0.99838424
RMSE force:  1218758.5751059367 RMSE angle:  0.011120585578638454
MAE force:  696410.8516383168 MAE angle:  0.007010881875191458
test on dataset2
force max: [2.97945469e+08 1.16089791e+08 3.68481496e+07] angle max:  1.7905628653536911
true force max: [6.20017763e+08 1.17792602e+08 3.72855729e+07] true angle max:  1.5707600376421085
R2 force:  0.9992998 R2 angle:  0.99976236
RMSE force:  697305.8668954101 RMSE angle:  0.0042373346827652755
MAE force:  274619.5860491808 MAE angle:  0.0018694899694359363


test on model  4 

test on dataset1
force max: [1.4514488e+08 1.0564064e+08 3.3544703e+07] angle max:  1.55751

true force max: [6.20017763e+08 1.17792602e+08 3.72855729e+07] true angle max:  1.5707600376421085
R2 force:  0.9990255 R2 angle:  0.9988614
RMSE force:  822631.163046364 RMSE angle:  0.009275655882001875
MAE force:  410729.7007498575 MAE angle:  0.00420445274963084


test on dataset1
force max: [2.00729784e+08 1.12654158e+08 3.44787242e+07] angle max:  1.5871191445430237
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.5533418615598973
R2 force:  0.9982339 R2 angle:  0.99890935
RMSE force:  1079853.4820096854 RMSE angle:  0.009136442052138788
MAE force:  656534.2277492279 MAE angle:  0.0059666399702353965
test on dataset2 

force max: [2.07651348e+08 1.16448454e+08 3.73033588e+07] angle max:  1.6374859289195143
true force max: [6.20017763e+08 1.17792602e+08 3.72855729e+07] true angle max:  1.5707600376421085
R2 force:  0.99920964 R2 angle:  0.9986969
RMSE force:  740863.7686359636 RMSE angle:  0.009922921724642847
MAE force:  402575.6519785934 MAE angl

normlization

In [11]:
with open('x_scaler_min_max.pkl', 'rb') as x_scaler_min_max_file:
    x_scaler_min_max = pickle.load(x_scaler_min_max_file)

with open('x_scaler_z_score.pkl', 'rb') as x_scaler_z_score_file:
    x_scaler_z_score = pickle.load(x_scaler_z_score_file)

with open('x_scaler_robust.pkl', 'rb') as x_scaler_robust_file:
    x_scaler_robust = pickle.load(x_scaler_robust_file)

In [12]:
model_list = ['Model_1_mse_600k_new_min_max.h5','Model_1_mae_600k_new_min_max.h5',
              'Model_1_mse_600k_new_z_score.h5','Model_1_mae_600k_new_z_score.h5',
              'Model_1_mse_600k_new_robust_scaled.h5','Model_1_mae_600k_new_robust_scaled.h5']

scaler_list = [x_scaler_min_max,x_scaler_z_score, x_scaler_robust]

In [13]:
for i in range(3):
    model_name = model_list[2*i]
    scaler = scaler_list[i]
    
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1 for mse')
    test_model_1e4_scaled(model,x1,y1,scaler)
    print('test on dataset2 for mse')
    test_model_1e4_scaled(model,x2,y2,scaler)
    print('\n')
    
    model_name = model_list[2*i + 1]
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1 for mae')
    test_model_1e4_scaled(model,x1,y1,scaler)
    print('test on dataset2 for mae')
    test_model_1e4_scaled(model,x2,y2,scaler)
    print('\n')

test on dataset1 for mse
force max: [40843850.13081318 27552776.09990391  2645628.16697529] angle max:  0.9793979842045658
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.5533418615598973
R2 force:  0.33582985 R2 angle:  -0.10631633
RMSE force:  20940950.953152157 RMSE angle:  0.29099004526982886
MAE force:  16171597.039000902 MAE angle:  0.24763471221764452
test on dataset2 for mse
force max: [40843850.13101918 27552776.10004285  2645628.1669886 ] angle max:  0.9793979842095057
true force max: [6.20017763e+08 1.17792602e+08 3.72855729e+07] true angle max:  1.5707600376421085
R2 force:  0.35142344 R2 angle:  -0.113003135
RMSE force:  21222716.42038443 RMSE angle:  0.2900023064600956
MAE force:  16257775.070225315 MAE angle:  0.2465176188833756


test on dataset1 for mae
force max: [37714069.63060499 21805144.03590785  1883066.6331024 ] angle max:  0.9769372472418556
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.55334

In [9]:
def unit_vector_scaling(data):
    norms = np.linalg.norm(data, axis=1)
    scaled_data = data / norms[:, np.newaxis]
    return scaled_data

def max_absolute_scaling(data):
    max_abs = np.max(np.abs(data), axis=0)
    scaled_data = data / max_abs
    return scaled_data

def no_scale(data):
    return data

In [10]:
def test_model_1e4_f(model,x,y,f):
    
    pred_factor = np.array([1e4,1e4,1e4,1e-4])
    x = f(x)
    
#     y_pred = model.predict(x, verbose=0) 
    y_pred = custom_predict(x, model)
    y_pred = y_pred * pred_factor
    
    kn = 2.0e8 
    kt = 0.5 * kn
    true_factor = np.array([kn,kn,kt,1])
    
    y_true = y * true_factor
    r2_f = R2(y_true[:,:3],y_pred[:,:3])
    r2_a = R2(y_true[:,-1],y_pred[:,-1])    
    
    print('force max:', np.max(y_pred[:,:3], axis = 0), 'angle max: ', np.max(y_pred[:,-1]))
    print('true force max:', np.max(y_true[:,:3], axis = 0), 'true angle max: ', np.max(y_true[:,-1]))
    
    print('R2 force: ', r2_f, 'R2 angle: ', r2_a)
    print('RMSE force: ', RMSE(y_true[:,:3],y_pred[:,:3]), 'RMSE angle: ', RMSE(y_true[:,-1],y_pred[:,-1]))    
    print('MAE force: ', MAE(y_true[:,:3],y_pred[:,:3]), 'MAE angle: ', MAE(y_true[:,-1],y_pred[:,-1]))

In [11]:
model_list = ['Model_1_mse_600k_new_normalized_unit_vector.h5','Model_1_mae_600k_new_normalized_unit_vector.h5',
              'Model_1_mse_600k_new_scaled_max_abs.h5','Model_1_mae_600k_new_scaled_max_abs.h5',
              'Model_1_mse_600k_new_noscaled.h5','Model_1_mae_600k_new_noscaled.h5']

f_list = [unit_vector_scaling,max_absolute_scaling,no_scale]

In [12]:
for i in range(3):
    model_name = model_list[2*i]
    f = f_list[i]
    
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1 for mse')
    test_model_1e4_f(model,x1,y1,f)
    print('test on dataset2 for mse')
    test_model_1e4_f(model,x2,y2,f)
    print('\n')
    
    model_name = model_list[2*i + 1]
    model=load_model(model_name, custom_objects={'R2': R2})
    print('test on dataset1 for mae')
    test_model_1e4_f(model,x1,y1,f)
    print('test on dataset2 for mae')
    test_model_1e4_f(model,x2,y2,f)
    print('\n')

test on dataset1 for mse
force max: [40841658.59706771 27550824.25008559  2643022.78093194] angle max:  0.9793987687519363
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.5533418615598973
R2 force:  0.33584 R2 angle:  -0.10631454
RMSE force:  20940791.707440674 RMSE angle:  0.29098980187593365
MAE force:  16171220.538103115 MAE angle:  0.2476345638869915
test on dataset2 for mse
force max: [40841658.60207418 27550824.25346275  2643022.78125621] angle max:  0.9793987688719841
true force max: [6.20017763e+08 1.17792602e+08 3.72855729e+07] true angle max:  1.5707600376421085
R2 force:  0.35142863 R2 angle:  -0.11300123
RMSE force:  21222632.26566855 RMSE angle:  0.2900020564170644
MAE force:  16257453.108924791 MAE angle:  0.24651746581706158


test on dataset1 for mae
force max: [37714686.26217917 21805986.70443982  1888179.25550421] angle max:  0.9771387575769205
true force max: [2.62044649e+08 1.14114918e+08 3.58215381e+07] true angle max:  1.553341861